In [1]:
def tensor_to_image(tensor):
    tensor = tensor.squeeze().permute(1, 2, 0)
    return tensor.detach().numpy()

In [2]:
import diffusers
import torch
import matplotlib.pyplot as plt

In [3]:
vq_model = diffusers.VQModel()
unet = diffusers.UNet2DModel(sample_size=64)
schedule = diffusers.DDPMScheduler(num_train_timesteps=1000)

In [4]:
test_input = torch.randn((1, 3, 128, 128))
encode = model.encoder(test_input)
decode = model.decoder(encode)

NameError: name 'model' is not defined

In [ ]:
encode.shape

In [ ]:
decode.shape

In [ ]:
plt.imshow(tensor_to_image(test_input))

In [ ]:
plt.imshow(tensor_to_image(encode))

In [ ]:
plt.imshow(tensor_to_image(decode))

In [ ]:
ldm = diffusers.LDMPipeline(vq_model, unet, schedule)

In [ ]:
ldm = ldm.to("cuda")

In [ ]:
image = ldm()

In [ ]:
plt.imshow(image.images[0])

In [ ]:
image.images[0].size

In [ ]:
test_input = torch.randn((1, 3, 64, 64))
vq_model.decoder(test_input).shape

In [ ]:
vq_model.decoder

In [ ]:
schedule.add_noise()

In [ ]:
import numpy as np
import PIL.Image as Image

In [ ]:
image = np.array(Image.open("../dataset/ffhq/thumbnails128x128/00000.png"))
image_tensor = torch.tensor(image).permute(2, 0, 1).unsqueeze(0)/255

In [ ]:
image_tensor

In [ ]:
plt.imshow(tensor_to_image(image_tensor))

In [ ]:
noise = torch.randn(image_tensor.shape)
noisy_image = schedule.add_noise(image_tensor, noise, torch.LongTensor([999]))

In [ ]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.Resize((128, 128)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

In [ ]:
img_a = Image.fromarray((image))
img_b = preprocess(img_a).unsqueeze(0)

In [ ]:
schedule = diffusers.DDIMScheduler(num_train_timesteps=1000)

In [ ]:
noise = torch.randn(img_b.shape)
noisy_image = schedule.add_noise(img_b, noise, torch.LongTensor([100]))

In [ ]:
noisy_image

In [ ]:
Image.fromarray(((noisy_image.permute(0, 2, 3, 1) + 1.0) * 127.5).type(torch.uint8).numpy()[0])

In [ ]:
noisy_image

In [ ]:
image_tensor

# Dataloader

In [1]:
import os
from torch.utils.data import Dataset, DataLoader

In [2]:
len(os.listdir("../dataset/ffhq/thumbnails128x128/"))

70001

In [3]:
os.listdir("../dataset/ffhq/thumbnails128x128/")

['00000.png',
 '00001.png',
 '00002.png',
 '00003.png',
 '00004.png',
 '00005.png',
 '00006.png',
 '00007.png',
 '00008.png',
 '00009.png',
 '00010.png',
 '00011.png',
 '00012.png',
 '00013.png',
 '00014.png',
 '00015.png',
 '00016.png',
 '00017.png',
 '00018.png',
 '00019.png',
 '00020.png',
 '00021.png',
 '00022.png',
 '00023.png',
 '00024.png',
 '00025.png',
 '00026.png',
 '00027.png',
 '00028.png',
 '00029.png',
 '00030.png',
 '00031.png',
 '00032.png',
 '00033.png',
 '00034.png',
 '00035.png',
 '00036.png',
 '00037.png',
 '00038.png',
 '00039.png',
 '00040.png',
 '00041.png',
 '00042.png',
 '00043.png',
 '00044.png',
 '00045.png',
 '00046.png',
 '00047.png',
 '00048.png',
 '00049.png',
 '00050.png',
 '00051.png',
 '00052.png',
 '00053.png',
 '00054.png',
 '00055.png',
 '00056.png',
 '00057.png',
 '00058.png',
 '00059.png',
 '00060.png',
 '00061.png',
 '00062.png',
 '00063.png',
 '00064.png',
 '00065.png',
 '00066.png',
 '00067.png',
 '00068.png',
 '00069.png',
 '00070.png',
 '0007

In [4]:
root_dir = "../dataset/ffhq/thumbnails128x128/"

In [22]:
import numpy as np
import PIL.Image as Image
import torchvision

In [6]:
image = np.array(Image.open("../dataset/ffhq/thumbnails128x128/00000.png"))

In [33]:
from torchvision import transforms

preprocess = transforms.Compose(
    [
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.5], [0.5]),
    ]
)

In [8]:
class ffhq_Dataset(Dataset):
    def __init__(self, root_dir, img_list, augmentation):
        self.augmentation = augmentation
        self.img_list = img_list
        self.root_dir = root_dir
        
    def __len__(self):
        return len(self.img_list)
    
    def __getitem__(self, idx):
        img = Image.open(root_dir + self.img_list[idx])
        return self.augmentation(img)
        

In [9]:
img_list = os.listdir("../dataset/ffhq/thumbnails128x128/")

In [10]:
ffhq_ds = ffhq_Dataset(root_dir, img_list, preprocess)

In [11]:
ffhq_ds[1]

tensor([[[-0.9373, -0.9294, -0.9451,  ..., -0.9843, -0.9843, -0.9843],
         [-0.9373, -0.9216, -0.9294,  ..., -0.9765, -0.9843, -0.9843],
         [-0.9373, -0.9373, -0.9373,  ..., -0.9765, -0.9765, -0.9843],
         ...,
         [ 0.5373,  0.5294,  0.5294,  ..., -0.1608, -0.1373, -0.1294],
         [ 0.5529,  0.5608,  0.5451,  ..., -0.1608, -0.1294, -0.1294],
         [ 0.6314,  0.6000,  0.6000,  ..., -0.1686, -0.1529, -0.1294]],

        [[-0.9765, -0.9765, -0.9765,  ..., -0.9843, -0.9843, -0.9843],
         [-0.9765, -0.9765, -0.9765,  ..., -0.9765, -0.9843, -0.9843],
         [-0.9765, -0.9765, -0.9765,  ..., -0.9765, -0.9765, -0.9843],
         ...,
         [-0.0980, -0.0902, -0.0510,  ..., -0.7961, -0.8039, -0.7882],
         [-0.0902, -0.0353, -0.0353,  ..., -0.8118, -0.8039, -0.7961],
         [-0.0039, -0.0275, -0.0196,  ..., -0.7961, -0.8039, -0.7961]],

        [[-1.0000, -1.0000, -1.0000,  ..., -1.0000, -1.0000, -1.0000],
         [-1.0000, -1.0000, -1.0000,  ..., -0

In [34]:
from dataclasses import dataclass

@dataclass
class TrainingConfig:
    image_size = 32  # the generated image resolution
    train_batch_size = 2
    eval_batch_size = 2  # how many images to sample during evaluation
    num_epochs = 50
    gradient_accumulation_steps = 1
    learning_rate = 1e-4
    lr_warmup_steps = 500
    save_image_epochs = 10
    save_model_epochs = 30
    mixed_precision = 'fp16'  # `no` for float32, `fp16` for automatic mixed precision
    output_dir = 'ddpm-butterflies-128'  # the model namy locally and on the HF Hub

    push_to_hub = True  # whether to upload the saved model to the HF Hub
    hub_private_repo = False  
    overwrite_output_dir = True  # overwrite the old model when re-running the notebook
    seed = 0

config = TrainingConfig()

In [35]:
# configs
batch_size=8

In [36]:
train_loader = DataLoader(ffhq_ds, batch_size=batch_size, shuffle=True)


#cifar 10
cifar_dataset = torchvision.datasets.CIFAR10(root= "../dataset/", download=True, transform=preprocess)
train_loader = DataLoader(cifar_dataset, batch_size=batch_size, shuffle=True)

Files already downloaded and verified


In [37]:
from diffusers import UNet2DModel


model = UNet2DModel(
    sample_size=config.image_size,  # the target image resolution
    in_channels=3,  # the number of input channels, 3 for RGB images
    out_channels=3,  # the number of output channels
    layers_per_block=2,  # how many ResNet layers to use per UNet block
    block_out_channels=(128, 128, 256, 256, 512, 512),  # the number of output channes for each UNet block
    down_block_types=( 
        "DownBlock2D",  # a regular ResNet downsampling block
        "DownBlock2D", 
        "DownBlock2D", 
        "DownBlock2D", 
        "AttnDownBlock2D",  # a ResNet downsampling block with spatial self-attention
        "DownBlock2D",
    ), 
    up_block_types=(
        "UpBlock2D",  # a regular ResNet upsampling block
        "AttnUpBlock2D",  # a ResNet upsampling block with spatial self-attention
        "UpBlock2D", 
        "UpBlock2D", 
        "UpBlock2D", 
        "UpBlock2D"  
      ),
)

In [38]:
import torch.nn.functional as F
import torch

# noise_pred = model(noisy_image, timesteps).sample
# loss = F.mse_loss(noise_pred, noise)

In [39]:
optimizer = torch.optim.AdamW(model.parameters(), lr=config.learning_rate)

In [40]:
from diffusers.optimization import get_cosine_schedule_with_warmup

lr_scheduler = get_cosine_schedule_with_warmup(
    optimizer=optimizer,
    num_warmup_steps=config.lr_warmup_steps,
    num_training_steps=(len(train_loader) * config.num_epochs),
)

In [41]:
from diffusers import DDPMScheduler

noise_scheduler = DDPMScheduler(num_train_timesteps=1000)

In [42]:
# train
from accelerate import Accelerator
from diffusers.hub_utils import init_git_repo, push_to_hub

from tqdm.auto import tqdm
import os

def train_loop(config, model, noise_scheduler, optimizer, train_dataloader, lr_scheduler):
    # Initialize accelerator and tensorboard logging
    accelerator = Accelerator(
        mixed_precision=config.mixed_precision,
        gradient_accumulation_steps=config.gradient_accumulation_steps,
        log_with="wandb",
        #logging_dir=os.path.join(config.output_dir, "logs")
    )
#     if accelerator.is_main_process:
#         if config.push_to_hub:
#             repo = init_git_repo(config, at_init=True)
    accelerator.init_trackers("train_example")
    
    # Prepare everything
    # There is no specific order to remember, you just need to unpack the 
    # objects in the same order you gave them to the prepare method.
    model, optimizer, train_dataloader, lr_scheduler = accelerator.prepare(
        model, optimizer, train_dataloader, lr_scheduler
    )
    
    global_step = 0

    # Now you train the model
    for epoch in range(config.num_epochs):
        progress_bar = tqdm(total=len(train_dataloader), disable=not accelerator.is_local_main_process)
        progress_bar.set_description(f"Epoch {epoch}")

        for step, batch in enumerate(train_dataloader):
            clean_images = batch[0]
            # Sample noise to add to the images
            noise = torch.randn(clean_images.shape).to(clean_images.device)
            bs = clean_images.shape[0]

            # Sample a random timestep for each image
            timesteps = torch.randint(0, noise_scheduler.num_train_timesteps, (bs,), device=clean_images.device).long()

            # Add noise to the clean images according to the noise magnitude at each timestep
            # (this is the forward diffusion process)
            noisy_images = noise_scheduler.add_noise(clean_images, noise, timesteps)
            
            with accelerator.accumulate(model):
                # Predict the noise residual
                noise_pred = model(noisy_images, timesteps, return_dict=False)[0]
                loss = F.mse_loss(noise_pred, noise)
                accelerator.backward(loss)

                accelerator.clip_grad_norm_(model.parameters(), 1.0)
                optimizer.step()
                lr_scheduler.step()
                optimizer.zero_grad()
            
            progress_bar.update(1)
            logs = {"loss": loss.detach().item(), "lr": lr_scheduler.get_last_lr()[0], "step": global_step}
            progress_bar.set_postfix(**logs)
            accelerator.log(logs, step=global_step)
            global_step += 1

        # After each epoch you optionally sample some demo images with evaluate() and save the model
        if accelerator.is_main_process:
            pipeline = DDPMPipeline(unet=accelerator.unwrap_model(model), scheduler=noise_scheduler)

            if (epoch + 1) % config.save_image_epochs == 0 or epoch == config.num_epochs - 1:
                evaluate(config, epoch, pipeline)

            if (epoch + 1) % config.save_model_epochs == 0 or epoch == config.num_epochs - 1:
                if config.push_to_hub:
                    push_to_hub(config, pipeline, repo, commit_message=f"Epoch {epoch}", blocking=True)
                else:
                    pipeline.save_pretrained(config.output_dir) 

In [43]:
from accelerate import notebook_launcher
args = (config, model, noise_scheduler, optimizer, train_loader, lr_scheduler)

notebook_launcher(train_loop, args, num_processes=1)

Launching training on one GPU.


wandb: wandb version 0.13.5 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


  0%|          | 0/6250 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [33]:
from accelerate import Accelerator

accelerator = Accelerator(log_with="all")
config = {
    "num_iterations": 5,
    "learning_rate": 1e-2,
    #"loss_function": str(my_loss_function),
}

accelerator.init_trackers("example_project", config=config)

TypeError: expected str, bytes or os.PathLike object, not NoneType